In [1]:
import os
from config import set_environment 
set_environment()

import logging
import sys
#logging.basicConfig(stream=sys.stdout, level=logging.INFO)
#logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Only for notebook
import nest_asyncio
nest_asyncio.apply()

In [2]:
from llama_index.core import Settings
from llama_index.core.evaluation import (
    CorrectnessEvaluator,
)
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.llms.cohere import Cohere
from llama_index.embeddings.cohere import CohereEmbedding
import pandas as pd
from datetime import datetime

In [3]:
eval_name = os.environ["EVAL_NAME"]
eval_results_file = os.environ["EVAL_RESULTS_FILE"]

In [4]:
evaluation_llm_family = os.environ["EVALUATION_LLM_FAMILY"]
evaluation_llm_model = os.environ["EVALUATION_LLM_MODEL"]

if evaluation_llm_family == "OPENAI":
    Settings.eval_llm = OpenAI(temperature=0, model=evaluation_llm_model)
elif evaluation_llm_family == "COHERE":
    Settings.eval_llm = Cohere(api_key=os.environ["COHERE_API_KEY"], model=evaluation_llm_model, temperature=0)

In [5]:
from llama_index.core.evaluation import (
    CorrectnessEvaluator,
)
eval_lidx_c = CorrectnessEvaluator(llm=Settings.eval_llm)

In [6]:
import pandas as pd

xls = pd.ExcelFile(eval_results_file)

# Load the "Responses" tab into a DataFrame
df_responses = pd.read_excel(xls, 'Responses')

# Define the function to evaluate responses
def evaluate_response(query, expected_answer, generated_answer):
    # Simple example of evaluation: check if expected answer is in the generated answer
    result = eval_lidx_c.evaluate( query=query, response=generated_answer, reference=expected_answer, )
    correctness = result.score
    feedback = result.feedback
    return correctness, feedback

# Apply the function to each row and create new columns
df_responses[['correctness_llm', 'correctness_feedback_llm']] = df_responses.apply(
    lambda row: evaluate_response(row['query'], row['expected_answer'], row['generated_answer']),
    axis=1, result_type='expand'
)

df_responses['correctness_human'] = df_responses['correctess_llm'] 
df_responses['correctness_feedback_human'] = "Agree with LLM assessment"
df_responses['run_date'] = datetime.today().strftime('%Y-%m-%d') 

df_responses.to_excel(eval_results_file, index=False)
